<a href="https://colab.research.google.com/github/JeromeHoen/Notifications_a_la_CNIL_de_violations_de_donnees_a_caractere_personnel/blob/master/CNIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [147]:
df = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/4c176588-a444-4dc7-b6bf-60390ae7e5be", encoding='utf-8', sep=';')
df.head()

,Date de réception de la notification,Secteur d'activité de l'organisme concerné,Natures de la violation,Nombre de personnes impactées,Typologie des données impactées,Données sensibles,Origine de l'incident,Causes de l'incident,Information des personnes
0,mars-19,"Activités spécialisées, scientifiques et techn...",Perte de la confidentialité,Entre 301 et 5000 personnes,Coordonnées (ex : adresse postale ou électroni...,NaN,Publication non volontaire d'informations,Acte interne accidentel,"Oui, les personnes ont été informées"
1,mars-19,Activités immobilières,Perte de la confidentialité,Entre 301 et 5000 personnes,"Etat civil (ex : nom, sexe, date de naissance,...",NaN,Données personnelles envoyées à un mauvais des...,Acte interne accidentel,Non déterminé pour le moment
2,mars-19,Administration publique,Perte de la confidentialité,Entre 6 et 50 personnes,"Etat civil (ex : nom, sexe, date de naissance,...",NaN,Données personnelles envoyées à un mauvais des...,Autre,"Oui, les personnes ont été informées"
3,mars-19,Activités financières et d''assurance,Perte de la confidentialité,Entre 0 et 5 personnes,"Etat civil (ex : nom, sexe, date de naissance,...",NaN,"Publication non volontaire d'informations,Donn...",Acte interne accidentel,"Non, mais elles le seront"
4,mars-19,Activités financières et d''assurance,Perte de la confidentialité,Entre 0 et 5 personnes,"Etat civil (ex : nom, sexe, date de naissance,...",NaN,Publication non volontaire d'informations,Acte interne accidentel,Non ils ne le seront pas


In [8]:
for col in df.columns:
    display(df[col].value_counts())

juin-18          322
janv-19          204
février 2019     173
juil-18          167
nov-18           157
oct-18           149
décembre 2018    128
août 2018        127
sept-18          103
mars-19          103
mai-18            17
Name: Date de réception de la notification, dtype: int64

Commerce ; réparation d''automobiles et de motocycles                                                                                                  241
Activités financières et d''assurance                                                                                                                  215
Activités spécialisées, scientifiques et techniques                                                                                                    215
Hébergement et restauration                                                                                                                            195
Administration publique                                                                                                                                151
Information et communication                                                                                                                           149
Autres activités de services                                          

Perte de la confidentialité                                                   1366
Perte de la confidentialité,Perte de la disponibilité                           82
Perte de la disponibilité                                                       76
Perte de la confidentialité,Perte de l'intégrité                                57
Perte de la confidentialité,Perte de l'intégrité,Perte de la disponibilité      34
Perte de l'intégrité                                                            22
Perte de l'intégrité,Perte de la disponibilité                                  13
Name: Natures de la violation, dtype: int64

Entre 301 et 5000 personnes    465
Entre 0 et 5 personnes         363
Entre 51 et 300 personnes      347
Entre 6 et 50 personnes        266
Plus de 5000 personnes         209
Name: Nombre de personnes impactées, dtype: int64

Etat civil (ex : nom, sexe, date de naissance, âge...),Coordonnées (ex : adresse postale ou électronique, numéros de téléphone fixe ou portable...)                                                                                                                                                                                                                                                                                                                                                                                                                                          220
Coordonnées (ex : adresse postale ou électronique, numéros de téléphone fixe ou portable...)                                                                                                                                                                                                                                                                                                                                           

Oui    156
Name: Données sensibles, dtype: int64

Piratage, logiciel malveillant (par exemple rançongiciel) et/ou hameçonnage                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       724
Autre                                                                                                                                                                                                                                                                                                                                                                                                                                                             

Acte externe malveillant                                                     855
Acte interne accidentel                                                      326
Inconnu                                                                      126
Autre                                                                         91
Acte externe accidentel                                                       88
Acte interne malveillant                                                      62
Acte interne accidentel,Autre                                                 20
Acte interne accidentel,Acte externe malveillant                              17
Acte interne malveillant,Acte externe malveillant                             16
Acte externe malveillant,Acte externe accidentel                              12
Acte externe malveillant,Inconnu                                              11
Acte externe malveillant,Autre                                                 6
Acte interne malveillant,Act

Non déterminé pour le moment            552
Non, mais elles le seront               489
Oui, les personnes ont été informées    411
Non ils ne le seront pas                198
Name: Information des personnes, dtype: int64

In [0]:
string_columns = df.select_dtypes(include='object').columns
for col in string_columns:
    df[col] = df[col].str.replace("''", "'").str.replace("’", "'")

In [0]:
df = df.fillna("Inconnu")

In [0]:
def series_to_count(series, split_character=",", ascending=False, return_dtype=None, skipna=True):
    """Return a pandas Series with each element counted after a split element-wise"""
    
    if not hasattr(series, "apply") or not hasattr(series, "name"):
        raise TypeError('Not a pandas Series')
    else:
        if split_character:
            tmp_series = series.apply(lambda s: s.split(split_character) if isinstance(s, str) else s)
        else:
            tmp_series = series.apply(lambda s: list(s) if isinstance(s, str) else s)
        count_dict = {}
        for row in tmp_series:
            if isinstance(row, list):
                for elt in row:
                    if elt in count_dict:
                        count_dict[elt] += 1
                    else:
                        count_dict[elt] = 1
            elif not skipna:
                if row in count_dict:
                    count_dict[row] += 1
                else:
                    count_dict[row] = 1
        return pd.Series(count_dict, index=sorted(count_dict, key=count_dict.get, reverse=not ascending), dtype=return_dtype, name=series.name)

In [100]:
violation = series_to_count(df['Natures de la violation'])
violation

Perte de la confidentialité    1539
Perte de la disponibilité       205
Perte de l'intégrité            126
Name: Natures de la violation, dtype: int64

In [0]:
def remove_text_inside_brackets(text, brackets="()[]"):
    count = [0] * (len(brackets) // 2) # count open/close brackets
    saved_chars = []
    for character in text:
        for i, b in enumerate(brackets):
            if character == b: # found bracket
                kind, is_close = divmod(i, 2)
                count[kind] += (-1)**is_close # `+1`: open, `-1`: close
                if count[kind] < 0: # unbalanced bracket
                    count[kind] = 0  # keep it
                else:  # found bracket to remove
                    break
        else: # character is not a [balanced] bracket
            if not any(count): # outside brackets
                saved_chars.append(character)
    return ''.join(saved_chars)

In [0]:
cleaned_typologie = df['Typologie des données impactées'].apply(remove_text_inside_brackets)

In [102]:
cleaned_typologie

0                                            Coordonnées 
1       Etat civil ,Coordonnées ,Données d'identificat...
2       Etat civil ,Coordonnées ,Données d'identificat...
3       Etat civil ,Coordonnées ,Données relatives à d...
4       Etat civil ,La violation concerne d'autres don...
5                                Etat civil ,Coordonnées 
6       Etat civil ,Coordonnées ,Données relatives à d...
7                                Etat civil ,Coordonnées 
8                  La violation concerne d'autres données
9       Etat civil ,Coordonnées ,Données d'identificat...
10                                                Inconnu
11                 La violation concerne d'autres données
12      Etat civil ,Coordonnées ,Données relatives à d...
13      Coordonnées ,Données relatives à des informati...
14          Etat civil ,Coordonnées ,Documents officiels 
15      Etat civil ,NIR ,Données d'identification ou d...
16      Etat civil ,NIR ,Coordonnées ,Données relative...
17      Etat c

In [103]:
typologie = series_to_count(cleaned_typologie)

new_index = [key.strip().capitalize() for key in typologie.index]
typologie.index = new_index

typologie

Etat civil                                                   1196
Coordonnées                                                  1182
Données relatives à des informations financières              632
Économiques                                                   632
Données d'identification ou d'accès                           469
La violation concerne d'autres données                        323
Nir                                                           159
Documents officiels                                           140
Données de localisation                                       101
Les données concernées ne sont pas connues pour le moment      85
Données relatives à des infractions                            35
Condamnations                                                  35
Mesures de sûreté                                              35
Inconnu                                                        12
Name: Typologie des données impactées, dtype: int64

In [104]:
cleaned_origine = df["Origine de l'incident"].apply(remove_text_inside_brackets)

origine = series_to_count(cleaned_origine)

new_index = [key.strip().capitalize() for key in origine.index]
origine.index = new_index

origine

Piratage                                                                                            759
Logiciel malveillant  et/ou hameçonnage                                                             759
Autre                                                                                               305
Données personnelles envoyées à un mauvais destinataire                                             224
Publication non volontaire d'informations                                                           179
Equipement perdu ou volé                                                                            161
Données de la mauvaise personne affichées sur le portail du client                                   57
Papier perdu                                                                                         49
Volé ou laissé accessible dans un endroit non sécurisé                                               49
Informations personnelles divulguées de façon verbale           

In [105]:
cleaned_cause = df["Causes de l'incident"].apply(remove_text_inside_brackets)

cause = series_to_count(cleaned_cause)

new_index = [key.strip().capitalize() for key in cause.index]
cause.index = new_index

cause

Acte externe malveillant    920
Acte interne accidentel     373
Inconnu                     144
Autre                       125
Acte externe accidentel     107
Acte interne malveillant     88
Name: Causes de l'incident, dtype: int64

In [0]:
french_month_to_num = {
    'jan': 1,
    'fév': 2,
    'fev': 2,
    'mar': 3,
    'avr': 4,
    'mai': 5,
    'juin': 6,
    'juil': 7,
    'aou': 8,
    'aoû': 8,
    'sep': 9,
    'oct': 10,
    'nov': 11,
    'déc': 12,
    'dec': 12
}

In [0]:
df['Mois'] = df['Date de réception de la notification'].str.lower().apply(
    lambda s: french_month_to_num[s[:4]]
              if s[:4] in french_month_to_num
              else french_month_to_num[s[:3]]
)

In [44]:
df['Mois'].value_counts()

6     322
1     204
2     173
7     167
11    157
10    149
12    128
8     127
9     103
3     103
5      17
Name: Mois, dtype: int64

In [50]:
import re
df['Année'] = df['Date de réception de la notification'].apply(lambda s: re.findall('\d+', s)[0]).astype(int)
df['Année'] = df['Année'].apply(lambda y: y + 2000 if y < 1000 else y)
df['Année'].value_counts()

2018    1170
2019     480
Name: Année, dtype: int64

In [137]:
col_tuples = []
for col in [violation, typologie, origine, cause]:
    for key in col.index:
        col_tuples.append("|".join((col.name, key)))
 
print(col_tuples)

['Natures de la violation|Perte de la confidentialité', 'Natures de la violation|Perte de la disponibilité', "Natures de la violation|Perte de l'intégrité", 'Typologie des données impactées|Etat civil', 'Typologie des données impactées|Coordonnées', 'Typologie des données impactées|Données relatives à des informations financières', 'Typologie des données impactées|Économiques', "Typologie des données impactées|Données d'identification ou d'accès", "Typologie des données impactées|La violation concerne d'autres données", 'Typologie des données impactées|Nir', 'Typologie des données impactées|Documents officiels', 'Typologie des données impactées|Données de localisation', 'Typologie des données impactées|Les données concernées ne sont pas connues pour le moment', 'Typologie des données impactées|Données relatives à des infractions', 'Typologie des données impactées|Condamnations', 'Typologie des données impactées|Mesures de sûreté', 'Typologie des données impactées|Inconnu', "Origine de 

In [0]:
df_multiindex = pd.DataFrame(index=df.index, columns=col_tuples)

In [139]:
for col_name in df_multiindex.columns:
    col, key = col_name.split("|")
    df_multiindex[col_name] = df[col].str.lower().str.contains(key.lower()).astype(np.uint8)
df_multiindex.head()

,Natures de la violation|Perte de la confidentialité,Natures de la violation|Perte de la disponibilité,Natures de la violation|Perte de l'intégrité,Typologie des données impactées|Etat civil,Typologie des données impactées|Coordonnées,Typologie des données impactées|Données relatives à des informations financières,Typologie des données impactées|Économiques,Typologie des données impactées|Données d'identification ou d'accès,Typologie des données impactées|La violation concerne d'autres données,Typologie des données impactées|Nir,Typologie des données impactées|Documents officiels,Typologie des données impactées|Données de localisation,Typologie des données impactées|Les données concernées ne sont pas connues pour le moment,Typologie des données impactées|Données relatives à des infractions,Typologie des données impactées|Condamnations,Typologie des données impactées|Mesures de sûreté,Typologie des données impactées|Inconnu,Origine de l'incident|Piratage,Origine de l'incident|Logiciel malveillant et/ou hameçonnage,Origine de l'incident|Autre,Origine de l'incident|Données personnelles envoyées à un mauvais destinataire,Origine de l'incident|Publication non volontaire d'informations,Origine de l'incident|Equipement perdu ou volé,Origine de l'incident|Données de la mauvaise personne affichées sur le portail du client,Origine de l'incident|Papier perdu,Origine de l'incident|Volé ou laissé accessible dans un endroit non sécurisé,Origine de l'incident|Informations personnelles divulguées de façon verbale,Origine de l'incident|Courrier perdu ou ouvert avant d'être retourné à l'envoyeur,Origine de l'incident|Mise au rebut de documents papier contenant des données personnelles sans destruction physique,Origine de l'incident|Mise au rebut d'appareils numériques contenant des données personnelles sans effacement sécurisé,Causes de l'incident|Acte externe malveillant,Causes de l'incident|Acte interne accidentel,Causes de l'incident|Inconnu,Causes de l'incident|Autre,Causes de l'incident|Acte externe accidentel,Causes de l'incident|Acte interne malveillant
0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
4,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
final_df = pd.DataFrame(index=df.index)
final_df[['Mois de réception de la notification',
          'Année de réception de la notification',
          "Secteur d'activité de l'organisme concerné",
          "Nombre de personnes impactées",
          "Données sensibles",
          "Information des personnes"]] = df[['Mois', 'Année', "Secteur d'activité de l'organisme concerné", "Nombre de personnes impactées", "Données sensibles", "Information des personnes"]]

In [141]:
final_df.head()

,Mois de réception de la notification,Année de réception de la notification,Secteur d'activité de l'organisme concerné,Nombre de personnes impactées,Données sensibles,Information des personnes
0,3,2019,"Activités spécialisées, scientifiques et techn...",Entre 301 et 5000 personnes,Inconnu,"Oui, les personnes ont été informées"
1,3,2019,Activités immobilières,Entre 301 et 5000 personnes,Inconnu,Non déterminé pour le moment
2,3,2019,Administration publique,Entre 6 et 50 personnes,Inconnu,"Oui, les personnes ont été informées"
3,3,2019,Activités financières et d'assurance,Entre 0 et 5 personnes,Inconnu,"Non, mais elles le seront"
4,3,2019,Activités financières et d'assurance,Entre 0 et 5 personnes,Inconnu,Non ils ne le seront pas


In [0]:
final_df = pd.concat((final_df, df_multiindex), axis=1)

In [0]:
import csv
final_df.to_csv("Notifications à la CNIL.csv", sep=",", encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [0]:
from google.colab import files
files.download("Notifications à la CNIL.csv")